#### Audio Classification using Random Forest classifier

##### Import the neccessary library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

##### Extract the feature from the audio

In [4]:
#Load the audio file
y,sr=librosa.load('D:/data/classification_data/data/coughs/fsd_1.wav')


In [73]:
# Extract features (example: MFCCs)
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)

In [74]:
print(mfccs)
print(spectral_centroid)
print(zero_crossing_rate)
print(spectral_rolloff)

[[-170.8301     -53.672573   -21.98235   ... -616.42126   -616.42126
  -616.42126  ]
 [  91.366165    65.35126     64.89467   ...    0.           0.
     0.       ]
 [ -24.218964   -18.114147   -26.62379   ...    0.           0.
     0.       ]
 ...
 [  -7.152582    -9.677477    -9.815548  ...    0.           0.
     0.       ]
 [   1.750874     1.3466088    5.0019717 ...    0.           0.
     0.       ]
 [ -13.508856    -8.628754    -4.6524906 ...    0.           0.
     0.       ]]
[[2535.3934934  3015.17805032 2988.13178543 2665.24704576 2713.5003522
  2783.53714203 2718.46267942 2629.95313502 2469.22805982 1858.89746726
  2035.99047763 2287.41943794 2304.1722508  2158.83880028 2005.54698215
  2245.82454662 2556.2955584  2645.83947381 2484.32233095 2437.4468949
  2480.67276987 2240.13100716 1785.81232126 2057.98855335 2202.0530778
  2221.64012345 2298.02864727 2156.59331274 2010.66251199 2315.16960636
  2493.85311945 2485.3866752  2297.20392726 2186.13709517 1946.29325311
  1761.1

In [25]:
# Concatenate features into a single feature vector
#features = np.column_stack([mfccs.mean(axis=1), spectral_centroid.mean(), zero_crossing_rate.mean()])
#print(features)
print((np.mean(mfccs, axis=1)))
print(zero_crossing_rate.mean())
print(spectral_centroid.mean())


[-3.9349091e+02  7.2570999e+01 -3.3433777e+01 -1.2432911e+01
 -2.0518051e-01 -5.0835176e+00 -1.4647078e+01 -1.4333396e+00
 -1.4500382e+01 -4.2926354e+00 -1.7296280e+01 -9.2658768e+00
 -3.9231362e+00 -3.5928541e-01 -1.0223149e+01 -6.0701241e+00
 -4.4255105e-01 -1.7500715e+00 -6.4876981e+00 -3.3872294e+00]
0.22471310314119172
2949.92275786351


#### Save the extracted feature results values in the lists

In [84]:
#1. Zero Crossing
#2. Spectral centroid
zc=[]
sc=[]
#3. Spectral Rolloff
sro=[]
#4. RMS Energy
rms=[]
label=[]

In [88]:
#cough- esc 1 to 40 and fsd- 1 to 256
#not_cough- esc 1 to 1960 and fsd 1 to 10000 
for i in range(1,41):
    filename=f"esc_{i}"
    #filename=f"fsd_{i}"
    # Load audio file and generate spectrogram
    y, sr = librosa.load('D:/data/classification_data/data/not_coughs/'+filename+".wav")

    #extract the feature
    spectral_centroid = (librosa.feature.spectral_centroid(y=y, sr=sr)).mean()
    zero_crossing_rate = (librosa.feature.zero_crossing_rate(y)).mean()
    spectral_rolloff = (librosa.feature.spectral_rolloff(y=y, sr=sr)).mean()
    rms_energy = (librosa.feature.rms(y=y)).mean()
    flag=0

    #Append the result
    sc.append(spectral_centroid)
    zc.append(zero_crossing_rate)
    rms.append(rms_energy)
    label.append(flag)
    sro.append(spectral_rolloff)


In [95]:
#Create a dataframe
x = pd.DataFrame({'Zero_crossing': zc, 'spectral_centriod': sc,'rms_energy': rms,'spectral_rolloff': sro})
print(x)
print(label)

     Zero_crossing  spectral_centriod  rms_energy  spectral_rolloff
0         0.239566        3163.854637    0.013364       5438.830020
1         0.039940        1218.510479    0.089246       2317.504938
2         0.069901        1992.682774    0.001401       4844.970703
3         0.095063        1749.604550    0.125050       3089.655762
4         0.194981        3292.252282    0.003525       5826.104577
..             ...                ...         ...               ...
587       0.352537        3460.821032    0.126654       4815.312703
588       0.066558        2645.862854    0.009184       4825.431315
589       0.016816         753.409721    0.153553       1367.956543
590       0.766382        6458.978217    0.060817       9658.389282
591       0.191836        3074.087776    0.008508       6519.027710

[592 rows x 4 columns]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

##### Normalization of the features

In [96]:
from sklearn.preprocessing import StandardScaler
# Initialize StandardScaler
scaler = StandardScaler()
x_scale = scaler.fit_transform(x)
x_scaled_df = pd.DataFrame(x_scale, columns=x.columns)
print(x_scaled_df)


     Zero_crossing  spectral_centriod  rms_energy  spectral_rolloff
0         0.978392           0.606538   -0.691136          0.346692
1        -0.889965          -1.008641    0.617948         -1.125500
2        -0.609550          -0.365862   -0.897510          0.066595
3        -0.374054          -0.567685    1.235615         -0.761311
4         0.561114           0.713144   -0.860871          0.529353
..             ...                ...         ...               ...
587       2.035727           0.853103    1.263288          0.052606
588      -0.640844           0.176460   -0.763243          0.057379
589      -1.106390          -1.394805    1.727328         -1.573361
590       5.909041           3.342411    0.127499          2.336874
591       0.531674           0.532006   -0.774897          0.856174

[592 rows x 4 columns]


In [98]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_scaled_df, label, test_size=0.2, random_state=42)

# Train Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on test data
y_pred = clf.predict(X_test)

# Evaluate classifier performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
training_accuracy=clf.score(X_train,y_train)

print("training_accuracy:",training_accuracy)
print("Accuracy1:", accuracy)
print("Classification Report:\n", report)

training_accuracy: 1.0
Accuracy: 0.7142857142857143
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.73      0.73        63
           1       0.70      0.70      0.70        56

    accuracy                           0.71       119
   macro avg       0.71      0.71      0.71       119
weighted avg       0.71      0.71      0.71       119

